In [1]:
from tdmq.client import Client

INFO:tdmq.client.client:Logging is active.


In [2]:
ngsi_msg = {
  "headers": [
    {
      "fiware-service": "tdm"
    },
    {
      "fiware-servicePath": "/cagliari/edge/meteo"
    },
    {
      "timestamp": 1531774294021
    }
  ],
  "body": {
    "attributes": [
      {
        "name": "altitude",
        "type": "float",
        "value": " "
      },
      {
        "name": "dateObserved",
        "type": "String",
        "value": "2018-07-16T20:51:33+00:00"
      },
      {
        "name": "location",
        "type": "geo:point",
        "value": "39.2479168, 9.1329701"
      },
      {
        "name": "timestamp",
        "type": "Integer",
        "value": "1531774293"
      },
      {
        "name": "windDirection",
        "type": "Float",
        "value": "174.545"
      },
      {
        "name": "windSpeed",
        "type": "Float",
        "value": "1.000"
      },
      {
        "name": "latitude",
        "type": "string",
        "value": "39.2479168"
      },
      {
        "name": "longitude",
        "type": "string",
        "value": "9.1329701"
      }
    ],
    "type": "WeatherObserved",
    "isPattern": "false",
    "id": "WeatherObserved:Edge-CFA703F4.esp8266-7806085.Davis"
  }
}

In [3]:
def create_src_desc(msg):
    headers = msg['headers']
    body = msg['body']
    attrs = dict((_['name'], _['value']) for _ in body['attributes'])
    desc = {}
    desc['id'] = body['id']
    # FIXME this is very fragile
    if body['type'] == 'WeatherObserved':
        desc['entity_type'] = 'PointWeatherObserver'
        desc['entity_category'] = 'Station'
    else:
        assert False
    desc['default_footprint'] = {
        'type': 'Point',
        'coordinates': [attrs['longitude'], attrs['latitude']]
    }
    desc['stationary'] = True
    desc['shape'] = []
    if body['id'].endswith(".Davis"):
        desc['controlledProperties'] = ["windDirection", "windSpeed"]
    else:
        assert False
    desc['description'] = {
        # random text
        "type": "multisensor",
        "brandName": "Davis",
        "modelName": "XXX",
        "manufacturerName": "Davis"
    }
    return desc

In [4]:
from datetime import datetime
def create_record(s, msg):
    headers = msg['headers']
    body = msg['body']
    attrs = dict((_['name'], _['value']) for _ in body['attributes'])
    t = datetime.fromtimestamp(float(attrs['timestamp']))
    data = dict((k, float(attrs[k])) for k in attrs if k in s.controlled_properties)
    return t, data

In [5]:
c = Client()

In [6]:
srcs = c.get_sources({'id': ngsi_msg['body']['id']})

In [7]:
if len(srcs) == 0:
    s = c.register_source(create_src_desc(ngsi_msg))
else:
    assert len(srcs) == 1
    s = srcs[0]
t, data = create_record(s, ngsi_msg) 
s.ingest(t, data)

In [8]:
ts = s.timeseries()

In [9]:
time, data = ts[:]

In [10]:
print(time)
print(data)

[datetime.datetime(2018, 7, 16, 20, 51, 33)]
{'windDirection': array([174.545]), 'windSpeed': array([1.])}
